In [ ]:
import numpy as np
from skimage import exposure
#from skimage.segmentation import quickshift, slic, watershed
import geopandas as gpd
import numpy as np
from osgeo import gdal
import scipy
import seaborn as sns
import pandas as pd
from osgeo import ogr
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

#import matplotlib.pyplot as plt
#import earthpy as py
#from sklearn import preprocessing
#from sklearn.preprocessing import MinMaxScaler
#from xgboost import XGBRFClassifier
#import xgboost as xgb

# import packages for hyperparameters tuning
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, cohen_kappa_score

import warnings
warnings.filterwarnings("ignore") 

In [ ]:
# COMPOSITE

naip_fn = r'C:\Users\10333076966\Documents\GEOPROCESSAMENTO\Temp\SUBSET\COMPOSITE.tif'
 
driverTiff = gdal.GetDriverByName('GTiff')
naip_ds = gdal.Open(naip_fn)
nbands = naip_ds.RasterCount
band_data = []

for i in range(1, nbands+1):
    band = naip_ds.GetRasterBand(i).ReadAsArray()
    #band_norm = preprocessing.normalize(band)

    #out_put = ((band-band.min())/ (band.max()-band.min()))

    #scaler = MinMaxScaler()
    #bands = scaler.fit_transform(band)
    band_data.append(band)
    
    #band_data.append(out_put)

    #band_data.append(band)

band_data = np.dstack(band_data)
p2, p98 = np.percentile(band_data, (2,98))

img = exposure.rescale_intensity(band_data, in_range=(p2, p98))


In [ ]:
# SEGMENTO
naip_seg = r'C:\Users\10333076966\Documents\GEOPROCESSAMENTO\Temp\SUBSET\SEGMENTACAO.tif'

driverTiff = gdal.GetDriverByName('GTiff')
naip_ds = gdal.Open(naip_seg)
segments = naip_ds.GetRasterBand(1).ReadAsArray()

In [ ]:
def segment_features(segment_pixels):
    SD = []
    features = []
    npixels, nbands = segment_pixels.shape
    for b in range(nbands):
        stats = scipy.stats.describe(band_data[:, i])
        SD.append(np.std(band_data[:, i]))
        band_stats = list(stats)[1:]
        if npixels == 1:
            # in this case the variance = nan, change it 0.0
            band_stats[3] = 0.0
        features += band_stats
        
    features.append(np.array(SD))
    return features

def segment_sizes(segment_pixels):
    area = segment_pixels.shape[0]*segment_pixels.shape[1]*30*30
    length = (segment_pixels.shape[0]*segment_pixels.shape[1]+segment_pixels.shape[0]*segment_pixels.shape[1])*30
    elongation = np.max(segment_pixels.shape)/np.min(segment_pixels.shape)
    compactness = np.sqrt(4*area)/length
    size_stats = [area,length,elongation,compactness]
    return size_stats

In [ ]:
features_sizes

In [ ]:
segment_ids = np.unique(segments)
objects = []
object_ids = []
for id in segment_ids:
    segment_pixels = img[segments == id]
    object_features = segment_features(segment_pixels)
    object_features_2 = segment_sizes(segment_pixels)
    objects.append(object_features)
    objects.append(object_features_2)
    object_ids.append(id)

In [ ]:
object_features_2

In [ ]:
# read shapefile to geopandas geodataframe
gdf = gpd.read_file(r'C:\Users\10333076966\Desktop\Landsat\RESULTADOS\2001_L7\Amostras_2001\teste_amostras_2001.shp')
# get names of land cover classes/labels
class_names = gdf['label'].unique()
# create a unique id (integer) for each land cover class/label
class_ids = np.arange(class_names.size) + 1
# create a pandas data frame of the labels and ids and save to csv
df = pd.DataFrame({'label': class_names, 'id': class_ids})
#df.to_csv(r'C:\Users\10333076966\Desktop\Landsat\RESULTADOS\1991_L5\Amostras_1991\class_lookup.csv')
# add a new column to geodatafame with the id for each class/label
gdf['id'] = gdf['label'].map(dict(zip(class_names, class_ids)))
 
# split the truth data into training and test data sets and save each to a new shapefile
gdf_train = gdf.sample(frac=0.8)  # 80% of observations assigned to training data (30% to test data)
gdf_test = gdf.drop(gdf_train.index)
# save training and test data to shapefiles
gdf_train.to_file(r'C:\Users\10333076966\Desktop\Landsat\RESULTADOS\2001_L7\Amostras_2001\train_data.shp')
gdf_test.to_file(r'C:\Users\10333076966\Desktop\Landsat\RESULTADOS\2001_L7\Amostras_2001\test_data.shp')
print(f'os labels serão: \n {df}' )

In [ ]:
train_fn = r'C:\Users\10333076966\Desktop\Landsat\RESULTADOS\2001_L7\Amostras_2001\train_data.shp'
train_ds = ogr.Open(train_fn)
lyr = train_ds.GetLayer()
# create a new raster layer in memory
driver = gdal.GetDriverByName('MEM')
target_ds = driver.Create('', naip_ds.RasterXSize, naip_ds.RasterYSize, 1, gdal.GDT_UInt16)
target_ds.SetGeoTransform(naip_ds.GetGeoTransform())
target_ds.SetProjection(naip_ds.GetProjection())
# rasterize the training points
options = ['ATTRIBUTE=id']
gdal.RasterizeLayer(target_ds, [1], lyr, options=options)
# retrieve the rasterized data and print basic stats
data = target_ds.GetRasterBand(1).ReadAsArray()

In [ ]:
# rasterized observation (truth, training and test) data
ground_truth = target_ds.GetRasterBand(1).ReadAsArray()

# get unique values (0 is the background, or no data, value so it is not included) for each land cover type

classes = np.unique(ground_truth)[1:]

# for each class (land cover type) record the associated segment IDs
segments_per_class = {}
for klass in classes:
    segments_of_class = segments[ground_truth == klass]
    segments_per_class[klass] = set(segments_of_class)
 
# make sure no segment ID represents more than one class
# intersection = set()
# accum = set()
# for class_segments in segments_per_class.values():
#     intersection |= accum.intersection(class_segments)
#     accum |= class_segments
# assert len(intersection) == 0, "Segment(s) represent multiple classes"

In [ ]:
train_img = np.copy(segments)
threshold = train_img.max() + 1  # make the threshold value greater than any land cover class value

# all pixels in training segments assigned value greater than threshold
for klass in classes:
    class_label = threshold + klass
    for segment_id in segments_per_class[klass]:
        train_img[train_img == segment_id] = class_label
 
# training segments receive land cover class value, all other segments 0
train_img[train_img <= threshold] = 0
train_img[train_img > threshold] -= threshold


# create objects and labels for training data
training_objects = []
training_labels = []
for klass in classes:
    class_train_object = [v for i, v in enumerate(objects) if segment_ids[i] in segments_per_class[klass]]
    training_labels += [klass] * len(class_train_object)
    training_objects += class_train_object


# TESTING DATA

In [ ]:
#testing data

test_fn = r'C:\Users\10333076966\Desktop\Landsat\RESULTADOS\2001_L7\Amostras_2001\test_data.shp'
test_ds = ogr.Open(test_fn)
lyr = test_ds.GetLayer()
# create a new raster layer in memory
driver = gdal.GetDriverByName('MEM')
target_test = driver.Create('', naip_ds.RasterXSize, naip_ds.RasterYSize, 1, gdal.GDT_UInt16)
target_test.SetGeoTransform(naip_ds.GetGeoTransform())
target_test.SetProjection(naip_ds.GetProjection())
# rasterize the training points
options = ['ATTRIBUTE=id']
gdal.RasterizeLayer(target_test, [1], lyr, options=options)
# retrieve the rasterized data and print basic stats


# rasterized observation (truth, training and test) data
ground_truth_test = target_test.GetRasterBand(1).ReadAsArray()

# get unique values (0 is the background, or no data, value so it is not included) for each land cover type
classes_test = np.unique(ground_truth_test)[1:]

# for each class (land cover type) record the associated segment IDs
segments_per_class_test = {}
for klass_test in classes_test:
    segments_of_class_test = segments[ground_truth_test == klass_test]
    segments_per_class_test[klass_test] = set(segments_of_class_test)
 

test_image = np.copy(segments)
threshold = test_image.max() + 1  # make the threshold value greater than any land cover class value

# all pixels in training segments assigned value greater than threshold
for klass_test in classes_test:
    class_label = threshold + klass_test
    for segment_id in segments_per_class_test[klass_test]:
        test_image[test_image == segment_id] = class_label
 
# training segments receive land cover class value, all other segments 0
test_image[test_image <= threshold] = 0
test_image[test_image > threshold] -= threshold


# create objects and labels for TESTING data
testing_objects = []
testing_labels = []
for klass_test in classes:
    class_test_object = [v for i, v in enumerate(objects) if segment_ids[i] in segments_per_class_test[klass_test]]
    testing_labels += [klass_test] * len(class_test_object)
    testing_objects += class_test_object
 

testing_labels = np.asarray(testing_labels, dtype=np.int32)
training_labels = np.asarray(training_labels, dtype=np.int32)

for i in range(len(training_labels)):
    if training_labels[i] == 6:
        training_labels[i] = 0


for i in range(len(testing_labels)):
    if testing_labels[i] == 6:
        testing_labels[i] = 0


print(f'o maximo de treino labels é: {np.max(training_labels)}' )
print(f'o minimo de treino labels é: {np.min(training_labels)}' )


print(f'o maximo de teste labels é: {np.max(testing_labels)}' )
print(f'o minimo de teste labels é: {np.min(testing_labels)}' )


# Random Forest Padrão

In [ ]:
# #random forest
# classifier = RandomForestClassifier(n_jobs=-1)  # setup random forest classifier
# classifier.fit(training_objects, training_labels)  # fit rf classifier
# predicted = classifier.predict(objects)  # predict with rf classifier

# Metrics RF

In [ ]:
# from sklearn.metrics import confusion_matrix,accuracy_score
# y_pred = classifier.predict(testing_objects)
# cm = confusion_matrix(testing_labels, y_pred)
# print(cm)
# accuracy_score(testing_labels,y_pred)

In [ ]:
# accuracies = cross_val_score(estimator=classifier,X=testing_objects,y=testing_labels,cv=10)
# print("Accuracy: {:.2f}%".format(accuracies.mean()*100))

# GRIDSEARCH CV

In [ ]:
# from sklearn.model_selection import GridSearchCV
# parameters = [{'n_estimators':[10,20,30,40,50,60,70],
#                'max_depth': [3,4,5,7],
#                'criterion':['entropy'],
#                'min_samples_split':[5,4,6,7,8],
#                'max_features':['auto', 'sqrt', 'log2']
#                }]
# grid_search = GridSearchCV(estimator= classifier,
#                            param_grid= parameters,
#                            scoring = 'accuracy',
#                            n_jobs = -1,
#                            verbose=2,
#                            cv = 10)

# grid_search.fit(training_objects,training_labels)
# best_accuracy = grid_search.best_score_
# best_parameters = grid_search.best_params_
# print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
# print("Best Parameters:", best_parameters)

# hyperopt

In [ ]:
space = {'criterion' : hp.choice('criterion', ['entropy','gini']),
         'max_depth' : hp.quniform('max_depth',10,1200,10),
         'max_features' : hp.choice('max_features',['auto','sqrt','log2',None]),
         'min_samples_leaf' : hp.uniform('min_samples_leaf',0,0.5),
         'min_samples_split' : hp.uniform('min_samples_split',0,1),
         'n_estimators': hp.choice('n_estimators',[10,50,350,550,750,1200,1300,1500])
}

def objective(space):
    model = RandomForestClassifier( criterion = space['criterion'],
                                    max_depth = int(space['max_depth']),
                                    max_features = space['max_features'],
                                    min_samples_leaf = space['min_samples_leaf'],
                                    min_samples_split = space['min_samples_split'],
                                    n_estimators = space['n_estimators']
    )
    
    accuracy = cross_val_score(model, training_objects,training_labels,cv=10).mean()
    return {'loss' : -accuracy , 'status' : STATUS_OK}

In [ ]:
trials = Trials()
best = fmin(fn = objective,
            space=space,
            algo= tpe.suggest,
            max_evals=80,
            trials=trials
           )
best

In [ ]:
crit = {0: 'entropy', 1: 'gini'}
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 10, 1: 50, 2: 350, 3: 550, 4: 750, 5: 1200, 6: 1300, 7: 1500}

print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

In [ ]:
trainedforest = RandomForestClassifier(criterion = crit[best['criterion']], max_depth = int(best['max_depth']), 
                                       max_features = feat[best['max_features']], 
                                       min_samples_leaf = best['min_samples_leaf'], 
                                       min_samples_split = best['min_samples_split'], 
                                       n_estimators = est[best['n_estimators']]).fit(training_objects,training_labels)

                                       
predictionforest = trainedforest.predict(testing_objects)
print(confusion_matrix(testing_labels,predictionforest))
print(f'A acurácia global é de : {round(accuracy_score(testing_labels,predictionforest),4)}')
print(classification_report(testing_labels,predictionforest))
acc5 = accuracy_score(testing_labels,predictionforest)
kappa = cohen_kappa_score(testing_labels, predictionforest)
print(f'O kappa é de : {round(kappa,4)}')

In [ ]:
predicted = trainedforest.predict(objects)
# create numpy array from rf classifiation and save to raster
clf = np.copy(segments)

for segment_id, klass in zip(segment_ids, predicted):
    clf[clf == segment_id] = klass
 
mask = np.sum(img, axis=2)  # this section masks no data values
mask[mask > 0.0] = 1.0
mask[mask == 0.0] = 1.0
#mask[mask == 0.0] = -1.0
clf = np.multiply(clf, mask)
clf[clf < 0] = -9999.0
 
 
clfds = driverTiff.Create(r'C:\Users\10333076966\Desktop\Landsat\RESULTADOS\2001_L7\CLASSIFIED_SNIC_2001_.tif', naip_ds.RasterXSize, naip_ds.RasterYSize,
                          1, gdal.GDT_Float32)  # this section saves to raster
clfds.SetGeoTransform(naip_ds.GetGeoTransform())
clfds.SetProjection(naip_ds.GetProjection())
clfds.GetRasterBand(1).SetNoDataValue(-9999.0)
clfds.GetRasterBand(1).WriteArray(clf)
clfds = None
 
print('Done!')

In [ ]:
s

# Accuracy

## RFECV (number of features)

In [ ]:
from sklearn.feature_selection import RFECV

# Init, fit
rfecv = RFECV(
    estimator=trainedforest,
    min_features_to_select=3,
    step=1,
    n_jobs=-1,
    scoring="r2",
    cv=5,
)

_ = rfecv.fit(img[ground_truth > 0], ground_truth[ground_truth > 0])

print(f"o número de features a serem selecionadas é de: {rfe.support_.sum()}")

## RFE (Recursive feature selection)

In [ ]:
## RFE 
rfe = RFE(estimator=trainedforest, n_features_to_select=6, step = 1)


rfe.fit(img[ground_truth > 0], ground_truth[ground_truth > 0])

for i in range(img.shape[2]):
	print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

In [ ]:
import numpy as np
from sklearn import metrics
 
# read original image to get info for raster dimensions
naip_fn = r'C:\Users\10333076966\Documents\Temp\TestePCA\CLASSIFICADOR\BSI_NDVI_B1B2B3B4_NDWI_subset.tif'
driverTiff = gdal.GetDriverByName('GTiff')
naip_ds = gdal.Open(naip_fn)
 
# rasterize test data for pixel-to-pixel comparison
test_fn = r'C:\Users\10333076966\Documents\Temp\IMAGENS_PARA_TESTE\Amostras\test_data.shp'
test_ds = ogr.Open(test_fn)
lyr = test_ds.GetLayer()
driver = gdal.GetDriverByName('MEM')
target_ds = driver.Create('', naip_ds.RasterXSize, naip_ds.RasterYSize, 1, gdal.GDT_UInt16)
target_ds.SetGeoTransform(naip_ds.GetGeoTransform())
target_ds.SetProjection(naip_ds.GetProjection())
options = ['ATTRIBUTE=id']
gdal.RasterizeLayer(target_ds, [1], lyr, options=options)
 
truth = target_ds.GetRasterBand(1).ReadAsArray()  # truth/test data array
 
pred_ds = gdal.Open(r'C:\Users\10333076966\Documents\Temp\TestePCA\classified_result_xgbost.tif')  
pred = pred_ds.GetRasterBand(1).ReadAsArray()  # predicted data array
idx = np.nonzero(truth) # get indices where truth/test has data values
cm = metrics.confusion_matrix(truth[idx], pred[idx])  # create a confusion matrix at the truth/test locations
scores = metrics.accuracy_score(truth[idx], pred[idx])

# pixel accuracy
print(cm)


print(scores)

In [ ]:
#level 2 scores